# MLOps stage 2 : Experimentation: Vertex AI Training for XGBoost with Hyperparameter Tuning

## Overview

This notebook demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. Here we are covering stage 2 : Vertex AI training for XGBoost with hyperparameter tuning.

## Objective

In this tutorial, you learn how to use Vertex AI Training for training a XGBoost custom model.

This tutorial uses the following Google Cloud ML services:
- Vertex AI Training
- Vertex AI Model resource
- The steps performed include:
- Vizier hyperparameter tuning

Training using a Python package.
- Report accuracy when hyperparameter tuning.
- Save the model artifacts to Cloud Storage using GCSFuse.
- Create a Vertex AI Model resource.

## Dataset

The dataset used in this example is the Synthetic Financial Fraud dataset from Kaggle. PaySim simulates mobile money transactions based on a sample of real transactions extracted from one month of financial logs from a mobile money service implemented in an African country. The original logs were provided by a multinational company, who is the provider of the mobile financial service which is currently running in more than 14 countries all around the world.

## Installations

In [36]:
import os
import IPython

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

In [37]:
![ ! -f "pip_installed" ] && pip3 install {USER_FLAG} --upgrade --quiet \
google-cloud-aiplatform \
google-cloud-pipeline-components \
kfp && touch pip_installed

## Restart Kernel

In [38]:
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Set up Project Information

In [6]:
PROJECT_ID = "bq-experiments-350102"
REGION = "us-central1"
BUCKET_NAME = "bq-experiments-fraud" 
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [10]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

## Initialize AIP SDK

In [11]:
import google.cloud.aiplatform as aip

In [12]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## Set Pre-built Containers

In [13]:
TRAIN_VERSION = "xgboost-cpu.1-1"
DEPLOY_VERSION = "xgboost-cpu.1-1"

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

## Set Machine Type

In [20]:
MACHINE_TYPE = "n1-standard"
VCPU = "16"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

Train machine type n1-standard-16


# XGBoost Training

Next, set up the package for XGBoost training.

## Training Package Description

**Package layout**
Before you start the training, you need look at how a Python package is assembled for a custom training job. When unarchived, the package contains the following directory/file layout.
- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
    - __init__.py
    - task.py

The files setup.cfg and setup.py are the instructions for installing the package into the operating environment of the Docker image.

The file trainer/task.py is the Python script for executing the custom training job. Note, when we referred to it in the worker pool specification, we replace the directory slash with a dot (trainer.task) and dropped the file suffix (.py).

## Create UUDI

In [21]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

## Package Assembly

The following cells will assemble the training package.

In [22]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'cloudml-hypertune',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Financial Fraud Classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: bryanfreeman@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

## Create the task script for the training package

Next, we need to create the task.py script for driving the training package. Some noteable steps include:

Command-line arguments:
- model-dir: The location to save the trained model. When using Vertex AI custom training, the location will be specified in the environment variable: AIP_MODEL_DIR,
- dataset_data_url: The location of the training data to download.
- dataset_labels_url: The location of the training labels to download.
- boost-rounds: Tunable hyperparameter

Data preprocessing (get_data()):
- Download the dataset and split into training and test.

Training (train_model()):
- Trains the model

Evaluation (evaluate_model()):
- Evaluates the model.
- If hyperparameter tuning, reports the metric for accuracy.

Model artifact saving
- Saves the model artifacts and evaluation metrics where the Cloud Storage location specified by model-dir.

In [23]:
%%writefile custom/trainer/task.py
import datetime
import os
import subprocess
import sys
import pandas as pd
import xgboost as xgb
import hypertune
import argparse
import logging
import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from google.cloud import bigquery

# SET UP TRAINING SCRIPT ARGUMENTS
parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument("--project-id", dest="project_id",
                    type=str, help="Project id for bigquery client.")
parser.add_argument("--bq-table", dest="bq_table",
                    type=str, help="Download url for the training data.")
parser.add_argument('--max-depth', dest="max_depth",
                    type=int, help='Max depth of XGB tree', default=3)
parser.add_argument('--learning-rate', dest="learning_rate",
                    type=float, help='Learning rate of XGB model', default=0.1)
args = parser.parse_args()

logging.getLogger().setLevel(logging.INFO)

# Function to retrieve data from BigQuery
def get_data():
    logging.info("Downloading training data from BigQuery: {}, {}".format(args.project_id, args.bq_table))
    logging.info("Creating BigQuery client")
    bqclient = bigquery.Client(project=args.project_id)
    
    logging.info("Loading table data")
    table = bigquery.TableReference.from_string(args.bq_table)
    rows = bqclient.list_rows(table)
    dataframe = rows.to_dataframe()
    
    logging.info("Preparing data for training")
    dataframe["isFraud"] = dataframe["isFraud"].astype(int)
    dataframe.drop(['nameOrig','nameDest','isFlaggedFraud'],axis=1,inplace=True)
    X = pd.concat([dataframe.drop('type', axis=1), pd.get_dummies(dataframe['type'])], axis=1)
    y = X[['isFraud']]
    X = X.drop(['isFraud'],axis=1)
    
    logging.info("Splitting data for training")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=42, shuffle=True)
    
    logging.info("Finishing get_data")
    return X_train, X_test, y_train, y_test

# Function to train the model
def train_model(X_train, y_train):
    logging.info("Start training ...")
    model = xgb.XGBClassifier(
            scale_pos_weight=734,
            max_depth=args.max_depth,
            learning_rate=args.learning_rate
    )
    model.fit(X_train, y_train)
    
    logging.info("Training completed")
    return model

# Function to evaluate the model
def evaluate_model(model, X_test, y_test):
    logging.info("Preparing test data ...")
    data_test = xgb.DMatrix(X_test)
    
    logging.info("Getting test predictions ...")
    y_pred = model.predict(X_test)
    
    logging.info("Evaluating predictions ...")
    f1 = f1_score(y_test, y_pred, average='weighted')
    logging.info(f"Evaluation completed with weighted f1 score: {f1}")

    logging.info("Report metric for hyperparameter tuning ...")
    hpt = hypertune.HyperTune()
    hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='f1_score',
        metric_value=f1
    )
    
    logging.info("Finishing ...")
    return f1

X_train, X_test, y_train, y_test = get_data()
model = train_model(X_train, y_train)
f1 = evaluate_model(model, X_test, y_test)
metric_dict = {'f1_score': f1}

# GCSFuse conversion
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)

# Export the classifier to a file
gcs_model_path = os.path.join(args.model_dir, 'model.bst')
logging.info("Saving model artifacts to {}". format(gcs_model_path))
model.save_model(gcs_model_path)

logging.info("Saving metrics to {}/metrics.json". format(args.model_dir))
gcs_metrics_path = os.path.join(args.model_dir, 'metrics.json')
with open(gcs_metrics_path, "w") as f:
    f.write(json.dumps(metric_dict))

Writing custom/trainer/task.py


## Store Training Script to Cloud Storage Bucket

In [24]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_fraud.tar.gz

custom/
custom/setup.py
custom/setup.cfg
custom/README.md
custom/PKG-INFO
custom/trainer/
custom/trainer/task.py
custom/trainer/__init__.py
Copying file://custom.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  2.1 KiB/  2.1 KiB]                                                
Operation completed over 1 objects/2.1 KiB.                                      


## Create Custom Training Job

In [26]:
DISPLAY_NAME = "fraud_" + UUID

job = aip.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_fraud.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

## Prepare Training Command Line Arguments

In [27]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, UUID)
BQ_TABLE = "bq-experiments-350102.synthetic_financial_fraud.fraud_data"

DIRECT = False
if DIRECT:
    CMDARGS = [
        "--project-id=" + PROJECT_ID,
        "--bq-table=" + BQ_TABLE,
        "--model_dir=" + MODEL_DIR,
    ]
else:
    CMDARGS = [
        "--project-id=" + PROJECT_ID,
        "--bq-table=" + BQ_TABLE,
    ]

## Run Custom Training Job

In [30]:
model = job.run(
        model_display_name="fraud_" + UUID,
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=False,
    )

model_path_to_deploy = MODEL_DIR

Training Output directory:
gs://bq-experiments-fraud/phk9joqs 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/1320634548676984832?project=402374189238
CustomPythonPackageTrainingJob projects/402374189238/locations/us-central1/trainingPipelines/1320634548676984832 current state:
PipelineState.PIPELINE_STATE_RUNNING
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/3125399322787577856?project=402374189238
CustomPythonPackageTrainingJob projects/402374189238/locations/us-central1/trainingPipelines/1320634548676984832 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomPythonPackageTrainingJob projects/402374189238/locations/us-central1/trainingPipelines/1320634548676984832 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomPythonPackageTrainingJob projects/402374189238/locations/us-central1/trainingPipelines/1320634548676984832 current state:
PipelineState.PIPELINE_STATE_RUNNING
Cus

## Wait for Training Job to Complete

In [31]:
model.wait()

## Prepare Machine Specification for Hyperparameter Tuning Job

In [32]:
machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

## Prepare Disk Specification

In [33]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

## Prepare Command Line Arguments

In [34]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, UUID)
BQ_TABLE = "bq-experiments-350102.synthetic_financial_fraud.fraud_data"
MAX_DEPTH = '3'
LEARNING_RATE = '0.1'

CMDARGS = [
    "--project-id=" + PROJECT_ID,
    "--bq-table=" + BQ_TABLE,
    "--max-depth=" + MAX_DEPTH,
    "--learning-rate=" + LEARNING_RATE,
    "--model-dir=" + MODEL_DIR,
]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_URI + "/trainer_fraud.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

## Create Custom Job

Use the class **CustomJob** to create a custom job, such as for hyperparameter tuning, with the following parameters:
- **display_name**: A human readable name for the custom job.
- **worker_pool_specs**: The specification for the corresponding VM instances.
- **base_output_dir:** The Cloud Storage location for storing the model artifacts.

In [35]:
DISPLAY_NAME = "fraud_" + UUID

job = aip.CustomJob(
    display_name=DISPLAY_NAME,
    worker_pool_specs=worker_pool_spec,
    base_output_dir=MODEL_DIR,
)

## Create a hyperparameter tuning job

Use the class **HyperparameterTuningJob** to create a hyperparameter tuning job, with the following parameters:
- **display_name:** A human readable name for the custom job.
- **custom_job:** The worker pool spec from this custom job applies to the CustomJobs created in all the trials.
- **metrics_spec:** The metrics to optimize. The dictionary key is the metric_id, which is reported by your training job, and the dictionary value is the optimization goal of the metric('minimize' or 'maximize').
- **parameter_spec:** The parameters to optimize. The dictionary key is the metric_id, which is passed into your training job as a command line key word argument, and the dictionary value is the parameter specification of the metric.
- **search_algorithm:** The search algorithm to use: grid, random and None. If None is specified, the Vizier service (Bayesian) is used.
- **max_trial_count:** The maximum number of trials to perform.

In [36]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hpt_job = aip.HyperparameterTuningJob(
    display_name=DISPLAY_NAME,
    custom_job=job,
    metric_spec={
        "f1_score": "maximize",
    },
    parameter_spec={
        "max-depth": hpt.IntegerParameterSpec(min=3, max=8, scale="linear"),
        "learning-rate": hpt.DoubleParameterSpec(min=0.01, max=0.1, scale='log'),
    },
    search_algorithm=None,
    max_trial_count=6,
    parallel_trial_count=1,
)

## Run Custom Hyperparameter Tuning Job

In [37]:
hpt_job.run()

Creating HyperparameterTuningJob
HyperparameterTuningJob created. Resource name: projects/402374189238/locations/us-central1/hyperparameterTuningJobs/269606985639395328
To use this HyperparameterTuningJob in another session:
hpt_job = aiplatform.HyperparameterTuningJob.get('projects/402374189238/locations/us-central1/hyperparameterTuningJobs/269606985639395328')
View HyperparameterTuningJob:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/269606985639395328?project=402374189238
HyperparameterTuningJob projects/402374189238/locations/us-central1/hyperparameterTuningJobs/269606985639395328 current state:
JobState.JOB_STATE_PENDING
HyperparameterTuningJob projects/402374189238/locations/us-central1/hyperparameterTuningJobs/269606985639395328 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJob projects/402374189238/locations/us-central1/hyperparameterTuningJobs/269606985639395328 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJob pro

## Display Job Trial Results

In [38]:
print(hpt_job.trials)

[id: "1"
state: SUCCEEDED
parameters {
  parameter_id: "learning-rate"
  value {
    number_value: 0.0316227766016838
  }
}
parameters {
  parameter_id: "max-depth"
  value {
    number_value: 6.0
  }
}
final_measurement {
  step_count: 1
  metrics {
    metric_id: "f1_score"
    value: 0.9925353145647853
  }
}
start_time {
  seconds: 1672087279
  nanos: 283876608
}
end_time {
  seconds: 1672088238
}
, id: "2"
state: SUCCEEDED
parameters {
  parameter_id: "learning-rate"
  value {
    number_value: 0.05246758891493698
  }
}
parameters {
  parameter_id: "max-depth"
  value {
    number_value: 8.0
  }
}
final_measurement {
  step_count: 1
  metrics {
    metric_id: "f1_score"
    value: 0.9959945800438604
  }
}
start_time {
  seconds: 1672088468
  nanos: 137930329
}
end_time {
  seconds: 1672089374
}
, id: "3"
state: SUCCEEDED
parameters {
  parameter_id: "learning-rate"
  value {
    number_value: 0.030192090366942398
  }
}
parameters {
  parameter_id: "max-depth"
  value {
    number_v

## Best Trial

In [39]:
best = (None, None, None, 0.0)
for trial in hpt_job.trials:
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

print(best)

('5', 0.1, 8.0, 0.997435473476749)
